In [6]:
!pip install -U sagemaker

In [4]:
import sagemaker

In [7]:
!docker

/bin/bash: docker: command not found


In [8]:

import sagemaker


In [9]:
!pip install -qU awscli boto3 sagemaker


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.31.3 which is incompatible.
python-language-server 0.31.7 requires jedi<0.16,>=0.14.1, but you have jedi 0.18.2 which is incompatible.
python-language-server 0.31.7 requires ujson<=1.35; platform_system != "Windows", but you have ujson 5.7.0 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jed

In [52]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()

In [11]:
role

'arn:aws:iam::105165742711:role/service-role/AmazonSageMaker-ExecutionRole-20230713T120695'

In [207]:
from time import gmtime, strftime

model_name = "hls-finetuning-model"
# model_url = "https://s3-us-west-2.amazonaws.com/2023-igarss-tutorial/data/{username}/{model_name}".format(username='manasubramanian1', model_name=model_name)
model_url = "s3://2023-igarss-tutorial-store/models/"
container = "105165742711.dkr.ecr.us-west-2.amazonaws.com/sagemaker-studio-d-wkcfdaa8gnhj:iksha-436"

print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + container)

container = {"Image": container, "ModelDataUrl": model_url, "Mode": "MultiModel", "MultiModelConfig": { "ModelCacheSetting": "Disabled" }}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: hls-finetuning-model
Model data Url: s3://2023-igarss-tutorial-store/models/
Container image: 105165742711.dkr.ecr.us-west-2.amazonaws.com/sagemaker-studio-d-wkcfdaa8gnhj:iksha-436
Model Arn: arn:aws:sagemaker:us-west-2:105165742711:model/hls-finetuning-model


In [208]:
endpoint_config_name = "hls-finetuning"
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.p3.2xlarge",
            "InitialInstanceCount": 2,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint config name: hls-finetuning
Endpoint config Arn: arn:aws:sagemaker:us-west-2:105165742711:endpoint-config/hls-finetuning


In [209]:
import time

endpoint_name = "hls-finetuning-endpoint"
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

Endpoint name: hls-finetuning-endpoint
Endpoint Arn: arn:aws:sagemaker:us-west-2:105165742711:endpoint/hls-finetuning-endpoint
Endpoint Status: Creating
Waiting for hls-finetuning-endpoint endpoint to be in service...


'finetuning-2023-07-15-21-07-03'

In [202]:
import json

with open('test.json', 'w') as f:
    json.dump({'file_name': 's3://2023-igarss-tutorial-store/data/burn_scars/subsetted_512x512_HLS.S30.T10SEH.2018190.v1.4_merged.tif'}, f)


In [210]:

# val = open('subsetted_512x512_HLS.S30.T10SEH.2018245.v1.4_merged.tif', 'rb').read()
val = open('test.json', 'rb').read()
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    TargetModel="gurung1_burn_scars.tar.gz",  # this is the rest of the S3 path where the model artifacts are located
    Body=val,
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "{
  "code": 500,
  "type": "InternalServerException",
  "message": "Failed to start workers"
}
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/hls-finetuning-endpoint in account 105165742711 for more information.

In [204]:
sm_client.delete_endpoint(EndpointName=endpoint_name)


{'ResponseMetadata': {'RequestId': '36fea155-8741-44b1-b938-0cfc4c2125ab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '36fea155-8741-44b1-b938-0cfc4c2125ab',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 16 Jul 2023 06:16:45 GMT'},
  'RetryAttempts': 0}}

In [205]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)



{'ResponseMetadata': {'RequestId': '4a8ec9a0-ad15-4032-b064-1f4954100f64',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4a8ec9a0-ad15-4032-b064-1f4954100f64',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 16 Jul 2023 06:16:45 GMT'},
  'RetryAttempts': 0}}

In [206]:
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '7b65081e-c722-4ab3-b4d0-ef95bbf05efc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b65081e-c722-4ab3-b4d0-ef95bbf05efc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 16 Jul 2023 06:16:45 GMT'},
  'RetryAttempts': 0}}